In [1]:
train_sentences = './data_tmp/train_sentences.pkl'
train_labels = './data_tmp/train_labels.pkl'
test_sentences = './data_tmp/dev_sentences.pkl'
test_labels = './data_tmp/dev_labels.pkl'
train_srcs = './data_tmp/train_srcs.pkl'
train_keys = './data_tmp/train_keys.pkl'
test_srcs = './data_tmp/test_srcs.pkl'
test_keys = './data_tmp/test_keys.pkl'

train_txt = './data_tmp/train(12809).txt'
test_txt = './data_tmp/test(2030).txt'


In [2]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '0,1,2,3'
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [3]:
import torch
print(torch.cuda.current_device())
print(torch.cuda.device_count())

0
1


In [4]:
import pickle

In [5]:
train_sentences = pickle.load(open(train_sentences,'rb'))
train_labels = pickle.load(open(train_labels,'rb'))
test_sentences = pickle.load(open(test_sentences,'rb'))
test_labels = pickle.load(open(test_labels,'rb'))

train_srcs = pickle.load(open(train_srcs,'rb'))
train_keys = pickle.load(open(train_keys,'rb'))
test_srcs = pickle.load(open(test_srcs,'rb'))
test_keys = pickle.load(open(test_keys,'rb'))

train_txt = open(train_txt, 'r', encoding = 'utf-8').read().split('\n\n')

train_tar = []
for u in train_txt:
    train_tar.append('[CLS] ' + u.split('\n')[1])

In [6]:
print(len(train_sentences))
print(len(test_sentences))
print(len(train_srcs))
print(len(test_srcs))
print(len(train_keys))
print(len(test_keys))
print(len(train_tar))

print(test_srcs[0])
      

13442
2300
12801
2031
12801
2031
12801
['weaned', 'off', 'vent', 'to', 'Continuous positive airway pressure', 'and', 'was', 'extubated', 'in', 'the', 'afternoon', 'on', '9-2', 'by', 'the', 'lung', 'team', '.']


In [7]:
print(train_srcs[11000])

['#', ')', '[unused4]', ':', 'Patient', 'has', 'history', 'of', 'heavy', 'alcohol', 'abuse', ',', 'and', 'reports', 'having', 'up', 'to', '15-20', 'beers', 'per', 'day', '.']


In [8]:
n = 0
for u in train_sentences:
    n+=1
    if 'Patient has history of heavy alcohol' in u[0]:
        break
print(n)

11548


In [9]:
    print(train_sentences[11547])

['# ) Alcoholism : Patient has history of heavy alcohol abuse , and reports having up to 15-20 beers per day .', 'Alcoholism : Alcohol use disorder ; Addiction to alcohol', '# ) Alcohol use disorder : Patient has history of heavy alcohol abuse , and reports having up to 15-20 beers per day .', '# ) Addiction to alcohol : Patient has history of heavy alcohol abuse , and reports having up to 15-20 beers per day .']


In [10]:
for i,u in enumerate(train_srcs):
    tmp = ['[CLS]']
    for j,v in enumerate(u):
        if v == '[unused4]':
#             tmp.append('[SEP]')
            tmp.append('[MASK]')
#             tmp.append('[SEP]')
        else:
            tmp.append(v)
    train_srcs[i] = tmp
    
cnt = 0
for i,u in enumerate(test_srcs):
    tmp = ['[CLS]']
    for j,v in enumerate(u):
        if v == '[unused4]':
            cnt += 1
#             tmp.append('[SEP]')
            tmp.append('[MASK]')
#             tmp.append('[SEP]')
        else:
            tmp.append(v)
    test_srcs[i] = tmp

print(cnt)
            
for i,u in enumerate(train_srcs):
    print(' '.join(u))
    if i>5:
        break
for i,u in enumerate(test_srcs):
    print(' '.join(u))
    if i>5:
        break
print(train_keys[:5])
print(test_keys[:5])

2300
[CLS] Stage 2 [MASK] : On sacrum and scrotum .
[CLS] Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , bicarbonate 20 and white blood cell 11.3 .
[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .
[CLS] Serum screens were negative for benzodiazepenes , barbituates and tricyclic antidepressants .
[CLS] lastname was admitted to the Neurosurgery service after a study revealed a break in her [MASK] catheter at the level of her neck .
[CLS] Her [MASK] had what was likely a resolving [MASK] .
[CLS] He was instructed to take plavix for only three months .
[CLS] weaned off vent to Continuous positive airway pressure and was extubated in the afternoon on 9-2 by the lung team .
[CLS] she was intubated and was resuscitation after 10-22 minutes of [MASK] arrest .
[CLS] he was given succinyl choline and etomidate for intubation .
[CLS] had post-op [MASK] and was transfused with appropriate effect .
[CLS] the patient was maintai

In [62]:
train_src_new = []
train_tar_new = []

for src, tar in zip(train_srcs, train_tar):
    src = ' '.join(src)
    src = src.split(' ')
    src_new = src
    if src_new[-1] !='.':
        src_new.append('.')
    tar = tar.split(' ')
    tar_new = []
    p = 0
    flag = 0
    
#     print(' '.join(src_new))
#     print(' '.join(tar))
    for i,u in enumerate(src):
        if u == '[MASK]':
            while p<len(tar) and tar[p]!=src[i-1]:
                tar_new.append(tar[p])
                p+=1
            if p<len(tar):
                tar_new.append(tar[p])
                p+=1
            tar_new.append('[SEP]')
            while p<len(tar) and tar[p]!=src[i+1]:
                tar_new.append(tar[p])
                p+=1
            tar_new.append('[SEP]')
    while p<len(tar):
        tar_new.append(tar[p])
        p+=1
#     print(' '.join(src_new))
#     print(' '.join(tar_new))
#     print('')
    train_src_new.append(src_new)
    train_tar_new.append(tar_new)
    


for i,u in enumerate(train_src_new):
    tmp = []
    for j,v in enumerate(u):
        if v == '[MASK]':
            tmp.append('[SEP]')
            tmp.append('[MASK]')
            tmp.append('[SEP]')
        else:
            tmp.append(v)
    train_src_new[i] = tmp
for i,u in enumerate(test_srcs):
    tmp = []
    for j,v in enumerate(u):
        if v == '[MASK]':
            tmp.append('[SEP]')
            tmp.append('[MASK]')
            tmp.append('[SEP]')
        else:
            tmp.append(v)
    test_srcs[i] = tmp
print(len(train_src_new))
print(len(train_tar_new))

        


12801
12801


In [63]:
print(test_srcs[0])

['[CLS]', 'weaned', 'off', 'vent', 'to', 'Continuous positive airway pressure', 'and', 'was', 'extubated', 'in', 'the', 'afternoon', 'on', '9-2', 'by', 'the', 'lung', 'team', '.']


In [64]:
from bert_embedding import BertEmbedding
import mxnet as mx
import numpy as np
bertembedding = BertEmbedding(ctx=mx.gpu(), batch_size=64, max_seq_length=64)

In [65]:
with open('./data_tmp/abbrs-all.pkl','rb') as f:
    abbrs = pickle.load(f)

In [66]:
train_embs = []
for i,u in enumerate(train_srcs):
    p = 0
    for v in u:
        if v== '[MASK]':
            p+=1
    if p!=len(train_keys[i]):
        print(u, train_keys[i])
    if i>30:
        break
    
test_embs = []
for i,u in enumerate(test_srcs):
    p = 0
    for v in u:
        if v== '[MASK]':
            p+=1
    if p!=len(test_keys[i]):
        print(u, test_keys[i])

In [67]:
train_input = []
for u in train_src_new:
    train_input.append(' '.join(u))

test_input = []
for u in test_srcs:
    test_input.append(' '.join(u))

train_output = []
for u in train_tar_new:
    train_output.append(' '.join(u))

print(train_input[:5])
print(train_output[:5])
print(test_input[:5])


['[CLS] Stage 2 [SEP] [MASK] [SEP] : On sacrum and scrotum .', '[CLS] Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , bicarbonate 20 and white blood cell 11.3 .', '[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .', '[CLS] Serum screens were negative for benzodiazepenes , barbituates and tricyclic antidepressants .', '[CLS] lastname was admitted to the Neurosurgery service after a study revealed a break in her [SEP] [MASK] [SEP] catheter at the level of her neck .']
['[CLS] Stage 2 [SEP] break in skin [SEP] : On sacrum and scrotum .', '[CLS] Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , Bicarbonate 20 and white blood cells 11.3 .', '[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .', '[CLS] Serum toxicology test were negative for benzodiazepenes , barbituates and tricyclic antidepressants .', '[CLS] Patient was admitted to the Neurosurgery servic

In [68]:
from transformers import BertTokenizer
from importlib import import_module
import random
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
model_name = 'bert'
x = import_module('models.' + model_name)
config = x.Config(16)
tokenizer = config.tokenizer

def tokenize(input_data):
    ids = []
    labels = []
    masks = []
    for i in tqdm(range(len(input_data))):
        words = tokenizer.tokenize(input_data[i])
        ids.append(words)
    
    ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in ids],
                          maxlen=64, dtype="long", value=0,
                              truncating="post", padding="post")
        
    return ids
 
train_src = tokenize(train_input)
test_src = tokenize(test_input)
train_tar = tokenize(train_output)
train_src_masks = [[float(i != 0.0) for i in ii] for ii in train_src]
test_src_masks = [[float(i != 0.0) for i in ii] for ii in test_src]
train_tar_masks = [[float(i != 0.0) for i in ii] for ii in train_tar]

print(len(train_src))
print(len(test_src))
print(len(train_tar))



pickle.dump(train_src, open("./data_tmp/train_src_23.pkl",'wb'))
pickle.dump(train_src_masks, open("./data_tmp/train_src_masks_23.pkl",'wb'))


pickle.dump(test_src, open("./data_tmp/test_src_23.pkl",'wb'))
pickle.dump(test_src_masks, open("./data_tmp/test_src_masks_23.pkl",'wb'))

pickle.dump(train_tar, open("./data_tmp/train_tar_23.pkl",'wb'))
pickle.dump(train_tar_masks, open("./data_tmp/train_tar_masks_23.pkl",'wb'))






Using TensorFlow backend.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12801/12801 [00:03<00:00, 3688.50it/s]


12801
2031
12801


In [69]:
from tqdm import tqdm
train_embs = []
for (srcs, keys) in tqdm(zip(train_srcs,train_keys)):
    key_embs = []
    for key in keys:
        emb_values = []
        for i, value in enumerate(abbrs[key]):
            if i>=14:
                break
            embs = bertembedding.embedding([value])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
        while len(emb_values)<14:
            embs = bertembedding.embedding(['[PAD] [PAD]'])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
            
        key_embs.append(emb_values)
    train_embs.append(key_embs)
#     if len(train_embs)>=250:
#         break
            

21it [00:10,  2.03it/s]


KeyboardInterrupt: 

In [ ]:
print(len(train_embs))

In [ ]:
pickle.dump(train_embs, open('./data_tmp/train_embs.pkl','wb'))


In [ ]:
train_embs = pickle.load(open('./data_tmp/train_embs.pkl','rb'))

In [ ]:
print(len(train_embs[0][0]))

In [ ]:

test_embs = []
for (srcs, keys) in tqdm(zip(test_srcs,test_keys)):
    key_embs = []
    for key in keys:
        emb_values = []
        for i, value in enumerate(abbrs[key]):
            if i>=14:
                break
            embs = bertembedding.embedding([value])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
        while len(emb_values)<14:
            embs = bertembedding.embedding(['[PAD] [PAD]'])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
            
        key_embs.append(emb_values)
    test_embs.append(key_embs)
#     if len(train_embs)>=250:
#         break
            

In [ ]:
pickle.dump(test_embs, open('./data_tmp/test_embs.pkl','wb'))


In [12]:
test_tars = pickle.load(open("./data_tmp/test_tars.pkl", 'rb'))
for u in test_tars:
    print(u)

['Weaned off vent to continuous positive airway pressure and was extubated in the afternoon on 9-2 by the lung specialist team .', 'Weaned off vent to continuous positive airway pressure and was extubated in the afternoon on 9-2 by the lung specialist team .', 'Weaned off vent to continuous positive airway pressure and was extubated in the afternoon on 9-2 by the lung specialist team .', 'Weaned off vent to continuous positive airway pressure and was extubated in the afternoon on 9-2 by the lung specialist team .']
['She was intubated and was waken up after 10-22 minutes of pulseless electrical activity arrest .', 'She was intubated and was waken up after 10-22 minutes of unresponsive pulse arrest .', 'You were intubated and was waken up after 10-22 minutes of pulseless electrical activity arrest .', 'You were intubated and was waken up after 10-22 minutes of unresponsive pulse arrest .']
['He was given succinyl choline and etomidate for intubation .', 'He was given succinyl choline an

['The patient was placed on Bilevel positive airway pressure for a period of time .', 'The patient was placed on device that helps with breathing for a period of time .', 'You were placed on Bilevel positive airway pressure for a period of time .', 'You were placed on device that helps with breathing for a period of time .']
['She tolerated the procedure well , was extubated and she was started on strict nothing by mouth after the procedure .', 'She tolerated the procedure well , was extubated and she was started on strict nothing by mouth after the procedure .', 'You tolerated the procedure well , was extubated and you were started on strict nothing by mouth after the procedure .', 'You tolerated the procedure well , was extubated and you were started on strict nothing by mouth after the procedure .']
['The combination of increased alcohol use and daily tylenol ( 2-3g for 3 weeks prior to admission ) , however , would likely be enough insult to cause acute chemical-driven liver damage

In [ ]:
'''
模型3的输入src：
She was intubated and was waken up after 10-22 minutes of [MASK] arrest

然后src经过tokenize，替换[MASK]的embedding等操作
最后得到的输出是一个list，里面是token的id：
[id_0, id_1, ... ,id_63]

上面这个list经过decode_sentence这个函数得到下面这个句子:
she was intubated and was after after after after 10 minutes of heart arrest arrest . 

但是目标句是这样的：
She was intubated and was waken up after 10-22 minutes of [SEP] pulseless electrical activity [SEP] arrest .

现在的模型是这样：
第二步的输入把src处理成了：
Weaned off vent to [SEP] [MASK] [SEP] and was extubated in the afternoon on 9-2 by the [SEP] [MASK] [SEP] team .
ground truth中[MASK]对应的部分也会被[SEP]包起来，
这样模型输出的结果里面会有[SEP]这样的分隔符把关键词包起来，我只会把输出结果里面被包起来的那部分对原句对应的位置做替换

'''